In [2]:
import keras as K
import os.path

import sys

sys.path.append("../src")
from artstat import util
import numpy as np

floatx = "float32"
K.backend.set_floatx(floatx)

Using TensorFlow backend.


In [3]:
from keras.layers import Dense, Embedding, Reshape, Concatenate, CuDNNLSTM, Lambda
from keras import backend as B
import tensorflow as tf

In [4]:
import imp
imp.reload(util)

<module 'artstat.util' from '../src/artstat/util.py'>

In [9]:
datadir = "/home/pmilovanov/hg/my/data/gallery-pr/all"

path_train = os.path.join(datadir, "train")
path_test = os.path.join(datadir, "test")

glove = "/home/pmilovanov/data/glove/glove.6B.50d.txt"
glove_dim = 50

In [10]:
words, vocab = util.load_vocab("../vocab.txt", 100000)

In [11]:
emb_matrix = util.load_embeddings(vocab, glove_dim, glove)

In [12]:
#np.savez("embmat_100K.npz", emb_matrix=emb_matrix)

In [13]:
print(emb_matrix.shape, len(vocab))

(100001, 50) 100000


In [14]:
imp.reload(util)

seqlen = 128
#X, Y, Xu, Yu = util.load_data_sequences(path_train, vocab, seqlen, stride)

X, Xu = util.load_data(path_train, vocab, seqlen)


100%|##########| 22860/22860 [00:33<00:00, 690.48it/s]


In [15]:
input_x = K.Input((seqlen,), dtype="int32", name="input_x")
input_xu = K.Input((seqlen,), dtype=floatx, name="input_xu")
#input_y = K.Input((seqlen,), dtype="int32", name="input_y")
#input_yu = K.Input((seqlen,), dtype=floatx, name="input_yu")
input_y = K.Input((1,), dtype="int32", name="input_y")
input_yu = K.Input((1,), dtype=floatx, name="input_yu")

def expander(id):
    return Lambda(lambda x: B.expand_dims(x), name="expander_" + id )


#reshaper_u  = K.layers.Reshape((seqlen,1))
#resh_xu = reshaper_u(input_xu)
#resh_yu = reshaper_u(input_yu)

resh_xu = expander("xu")(input_xu)
resh_yu = expander("yu")(input_yu)

dim = emb_matrix.shape[1] + 1

L_emb1 = K.layers.Embedding(emb_matrix.shape[0], 
                           emb_matrix.shape[1], 
                           input_length=seqlen, 
                          trainable=False,
                           weights=[emb_matrix],
                           name="embedding1")


L_emb2 = K.layers.Embedding(emb_matrix.shape[0], 
                           emb_matrix.shape[1], 
                           input_length=1, 
                          trainable=False,
                           weights=[emb_matrix],
                           name="embedding2")


In [17]:
emb_x = L_emb1(input_x)
emb_y = L_emb2(input_y)

In [18]:
Lconcat = K.layers.Concatenate()
concat_x = Lconcat([emb_x, resh_xu])
concat_y = Concatenate()([emb_y, resh_yu])

In [19]:
def realdot(a,b):
    d = K.layers.multiply([a, b])
    return B.sum(d, axis=-1)

def dotlayer(x):
    a,b = x
    return realdot(a,b)

def cossim(x):
    a,b = x
    p = realdot(a,b)
    an = B.sqrt(realdot(a,a) + B.epsilon())
    bn = B.sqrt(realdot(b,b) + B.epsilon())
    return tf.divide(p, K.layers.multiply([an, bn]) + B.epsilon())



In [21]:
lstm =  K.layers.CuDNNLSTM(256, return_sequences=True, name='lstm1')(concat_x)
print(lstm.shape)
lstm = K.layers.BatchNormalization()(lstm)
lstm =  K.layers.CuDNNLSTM(256, return_sequences=False, name='lstm2')(lstm)
#lstm = K.layers.BatchNormalization()(lstm)
#lstm =  K.layers.CuDNNLSTM(128, return_sequences=False, name='lstm3')(lstm)
#lstm = K.layers.CuDNNGRU(256, return_sequences=True, name='gru2')(lstm)
#lstm = K.layers.BatchNormalization()(lstm)
#lstm = K.layers.CuDNNLSTM(256, return_sequences=False, name='gru3')(lstm)
#lstm = L_expander(lstm)
dense = K.layers.BatchNormalization()(lstm)

def candidate_gen(inp, layername):
    out = K.layers.Dense(dim, activation="relu", name=layername+"_1")(inp)
    out = K.layers.BatchNormalization()(out)
#    out = K.layers.Dense(dim, activation="relu", name=layername+"_2")(out)
#    out = K.layers.BatchNormalization()(out)
#    out = K.layers.Dense(dim, activation="relu", name=layername+"_3")(out)
#    out = K.layers.BatchNormalization()(out)
    out = K.layers.Dense(dim, activation="relu", name=layername+"_4")(out)
    out = K.layers.BatchNormalization()(out)
    out = K.layers.Dense(dim, activation="sigmoid", name=layername+"_last")(out)
    out = K.layers.BatchNormalization()(out)
    return out

out1 = candidate_gen(dense, "out1")
out2 = candidate_gen(dense, "out2")
out3 = candidate_gen(dense, "out3")

L_cossim_expand = expander("candidate_cossim")
L_cossim_out = Lambda(cossim, name="cossim-out123")
cossim_out_12 = L_cossim_expand(L_cossim_out([out1, out2]))
cossim_out_13 = L_cossim_expand(L_cossim_out([out1, out3]))
cossim_out_23 = L_cossim_expand(L_cossim_out([out2, out3]))

cossim_out_all = Concatenate(name="concat_cossim_out_all")([cossim_out_12,
                                                           cossim_out_13,
                                                           cossim_out_23])
#print("cossim_out_all", cossim_out_all.shape)
def candidates_proximity_penalty_fn(x):
    threshold = 0.9
    coeff = 20.0
    penalty = B.exp(coeff * B.maximum(0.0, x - threshold)) - 1.0
    penalty = B.max(penalty, axis=-1, keepdims=True)
    return penalty
    
candidates_proximity_penalty = Lambda(candidates_proximity_penalty_fn)(cossim_out_all)

allout = K.layers.Concatenate(name="concat_out")([out1, out2, out3, dense])

#lstm3 =  K.layers.CuDNNLSTM(256, return_sequences=True, name='lstm3')(allout)

#probs = K.layers.Dense(dim, activation="relu", name="probabilities_intermediate_1")(allout)
#probs = K.layers.BatchNormalization()(probs)
#probs = K.layers.Dense(dim, activation="relu", name="probabilities_intermediate_2")(probs)
#probs = K.layers.BatchNormalization()(probs)
probs = K.layers.Dense(3, activation="softmax", name="probabilities")(allout)

L_cossim = Lambda(cossim, name="cosine_similarity")
prox1 = L_cossim([concat_y, out1])
prox1 = expander("prox1")(prox1)
prox2 = L_cossim([concat_y, out2])
prox2 = expander("prox2")(prox2)
prox3 = L_cossim([concat_y, out3])
prox3 = expander("prox3")(prox3)

allprox = Concatenate(name="concat_prox")([prox1, prox2, prox3])
final1 = K.layers.Lambda(dotlayer, name="final1")([probs, allprox])

def final2_fun(x):
    threshold = 0.9
    coeff= 20.0
    f1, p, cand_prox_pen = x
    probs_penalty = B.exp(coeff * B.maximum(0.0, p - threshold)) - 1.0
    print("p2", probs_penalty.shape)
    probs_penalty = B.max(probs_penalty, axis=-1, keepdims=True)
    print(probs_penalty.shape)
    return B.abs(1.0 - f1) + probs_penalty + cand_prox_pen

final2 = Lambda(final2_fun, name="final2")([final1, 
                                            probs, candidates_proximity_penalty])
print("probs", probs.shape)
print(prox3.shape)
print(allout.shape)
print(allprox.shape)
print("final1", final1.shape)
print("final2", final2.shape)



(?, 128, 256)
p2 (?, 3)
(?, 1)
p2 (?, 3)
(?, 1)
probs (?, 3)
(?, 1, 1)
(?, 409)
(?, 1, 3)
final1 (?, 1)
final2 (?, 1)


In [22]:
model = K.Model(inputs=[input_x, input_xu, input_y, input_yu],
               outputs=[final2])

In [23]:
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_x (InputLayer)            (None, 128)          0                                            
__________________________________________________________________________________________________
input_xu (InputLayer)           (None, 128)          0                                            
__________________________________________________________________________________________________
embedding1 (Embedding)          (None, 128, 50)      5000050     input_x[0][0]                    
__________________________________________________________________________________________________
expander_xu (Lambda)            (None, 128, 1)       0           input_xu[0][0]                   
__________________________________________________________________________________________________
concatenat

In [27]:
opt = K.optimizers.Adam(lr=0.01)
model.compile(opt, loss='mean_absolute_error')

In [28]:
imp.reload(util)
batch_size=2048
train_seq = util.SpecialSequence(X, Xu, seqlen, batch_size)

[bX, bXu, bY, bYu], [bYfake] = train_seq[250]

print(bX.shape, bXu.shape, bY.shape, bYu.shape, bYfake.shape)

(2048, 128) (2048, 128) (2048, 1) (2048, 1) (2048, 1)


In [29]:
#model.fit([X, Xu, Y, Yu], [Yfake], batch_size=512, epochs=1000000)

model.fit_generator(train_seq, steps_per_epoch=int(len(train_seq)/(batch_size*seqlen)), epochs=1000000)

Epoch 1/1000000
66/66 [==============================] - 42s 634ms/step - loss: 1.0167
Epoch 2/1000000
66/66 [==============================] - 24s 359ms/step - loss: 0.6424
Epoch 3/1000000
66/66 [==============================] - 24s 361ms/step - loss: 0.5920
Epoch 4/1000000
66/66 [==============================] - 24s 364ms/step - loss: 0.5432
Epoch 5/1000000
66/66 [==============================] - 24s 363ms/step - loss: 0.5731
Epoch 6/1000000
 9/66 [===>..........................] - ETA: 21s - loss: 0.5699

KeyboardInterrupt: 

------------------------

In [30]:
predictor = K.Model([input_x, input_xu], [out1, out2, out3, probs])
predictor.compile(opt, loss="mean_absolute_error")

In [32]:
from sklearn import preprocessing as pp
norm_emb_matrix =  pp.normalize(L_emb1.get_weights()[0], axis=1)

In [33]:
len(X)

17676455

In [35]:
t2s = util.Text2Seq(vocab)

text = """
Lolita, light of my life, fire of my loins. My sin, my soul. Lo-lee-ta: the tip of the tongue taking a trip of three steps down the palate to tap, at three, on the teeth. Lo. Lee. Ta. She was Lo, plain Lo, in the morning, standing four feet ten in one sock. She was Lola in slacks. She was Dolly at school. She was Dolores on the dotted line. But in my arms she was always Lolita. Did she have a precursor? She did, indeed she did. In point of fact, there might have been no Lolita at all had I not loved, one summer, an initial girl-child. In a princedom by the sea. Oh when? About as many years before Lolita was born as my age was that summer. You can always count on a murderer for a fancy prose style. Ladies and gentlemen of the jury, exhibit number one is what the seraphs, the misinformed, simple, noble-winged seraphs, envied. Look at this tangle of thorns."""

sx, sxu = t2s.toseq(text)

if len(sx) > seqlen:
    sx = sx[-seqlen:]
    sxu = sxu[-seqlen:]

print(len(sx))
print(sx)
print(sxu)



128
[50, 25548, 22, 2, 23113, 816, 3, 692, 6, 496, 6680, 136, 50, 469, 32880, 3, 14700, 136, 56, 7, 14716, 90, 290, 1558, 1, 3354, 136, 1558, 3, 46, 301, 4, 868, 1, 276, 676, 56, 64, 265, 32880, 24, 80, 271, 86, 69, 9576, 1, 57, 1068, 1, 27, 3268, 7343, 8, 3578, 3, 46, 7, 0, 15, 2, 2090, 3, 7437, 238, 90, 1074, 16, 166, 111, 457, 32880, 50, 279, 16, 496, 902, 50, 23, 1068, 3, 744, 85, 469, 9261, 22, 7, 39117, 21, 7, 25552, 13257, 951, 3, 27152, 5, 51153, 4, 2, 1224, 1, 1493, 401, 57, 13, 123, 2, 0, 1, 2, 0, 1, 1262, 1, 17326, 8, 35712, 0, 1, 85260, 3, 6177, 24, 42, 28003, 4, 52218, 3]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [38]:
i = 99999


gen = X[i:i+seqlen].tolist()
genu = X[i:i+seqlen].tolist()

#gen = [0]*(seqlen - len(sx)) + sx
#genu = [0]*(seqlen - len(sxu)) + sxu


tX = np.zeros((1, seqlen), dtype="int32")
tXu = np.zeros((1, seqlen), dtype=floatx)
results = []

def vec2word(x):
    we = np.transpose(pp.normalize(np.reshape(x[0, :glove_dim], (1,glove_dim))))
    idx = np.argmax(np.matmul(norm_emb_matrix, we))
    return idx

iterations = 128

allcandidates = []
for j in range(iterations):
    tX[0,:] = np.array(gen[-seqlen:], "int32")
    tXu[0,:] = np.array(genu[-seqlen:], floatx)
    #print(tX)
    xv, xuv = 0, 0.0
    yhat = predictor.predict([tX, tXu])
    
#    print(yhat[0].shape)
#    break
    
    #for qq in yhat:
    #    print(qq.shape)
    #print(yhat)
    #break
    vecid = np.random.choice(3, p=yhat[3][0])
    z = yhat[vecid]
    
    
    candidates=[]
    for u,c in enumerate(yhat[:3]):
        cid = vec2word(c)
        candidates.append((words[cid], yhat[3][0,u]))
    allcandidates.append(candidates)
#    print(candidates)
    
    if False:# z[0, -1, -1] > 0.9:
        xuv = 1.0
        results.append("<UNK>")
    else:
#        we = np.transpose(pp.normalize(np.reshape(z[0, :300], (1,300))))
#        print(np.linalg.norm(we))
#        scores = np.matmul(norm_emb_matrix, we)
#        print(scores)
#        print(np.min(scores))
#        break
#        xv = np.argmax(np.matmul(norm_emb_matrix, we))
        xv = vec2word(z)
        if xv:
            results.append(words[xv])
        else:
            results.append("<EMPTY>")
    gen.append(xv)
    genu.append(xuv)
    sys.stdout.write(results[-1]+" ")
    sys.stdout.flush()
    
    
for cc in allcandidates:
    print("| %10s %.2f | %10s %.2f | %10s %.2f |" %
         (cc[0][0], cc[0][1],
         cc[1][0], cc[1][1],
         cc[2][0], cc[2][1]))

arde arde arde arde arde arde arde arde arde arde arde arde arde arde arde arde arde arde arde arde arde arde arde arde ugm ugm arde arde arde arde arde arde arde arde arde arde arde arde arde arde arde arde arde arde arde arde arde arde arde arde arde arde arde arde spectre ugm ugm ugm ugm reintroduction 4002 ugm ugm ugm reintroduction arde peintre collet arde peintre arde arde peintre arde peintre collet peintre collet peintre collet arde peintre charlatan peintre charlatan peintre arde peintre traitor arde arde traitor traitor traitor arde traitor arde arde traitor traitor traitor traitor arde traitor traitor traitor traitor arde arde traitor traitor traitor traitor arde traitor tyrant etoy arde arde traitor tyrant treason tyrant treason arde arde tyrant tyrant |       arde 1.00 |        5cm 0.00 |      blend 0.00 |
|       arde 1.00 |        5cm 0.00 |      makes 0.00 |
|       arde 1.00 |        5cm 0.00 |    quality 0.00 |
|       arde 1.00 |        5cm 0.00 |      makes 0.00 |
|

In [51]:
print(np.min(z), np.max(z))

-3.9707959 11.670498


In [92]:
from sklearn import preprocessing

In [93]:
norm_emb_matrix =  preprocessing.normalize(emb_matrix, axis=1)

In [94]:
zn = preprocessing.normalize(z[0,:], axis=1)


In [95]:
we = zn[-1, :300]

In [96]:
scores = np.matmul(norm_emb_matrix, we)

In [97]:
idx = np.argmax(scores)
print(idx)
word = words[idx]

23


In [98]:
word

'at'

In [84]:
words[2]

'.'